In [1]:
import requests
import pandas as pd

In [2]:
def ask_agent(prompt: str, url="http://localhost:8000/query"):
    response = requests.post(
        url,
        json={"query": prompt}
    )
    result = response.json()
    
    print("🧠 Prompt:", result["query"])
    print("📝 SQL:", result["sql"])
    
    return pd.DataFrame(result["results"])

In [3]:
ask_agent("Find 10 entries where MAT includes Ti and DSC includes alloys")

🧠 Prompt: Find 10 entries where MAT includes Ti and DSC includes alloys
📝 SQL: SELECT * FROM entries WHERE 'Ti' = ANY(mat) AND 'alloys' = ANY(dsc) LIMIT 10;


""


#### Search was empty so updated the query to be more forgiving and we get the following:

In [4]:
ask_agent("Find 10 entries where MAT includes Ti and DSC includes alloys")

🧠 Prompt: Find 10 entries where MAT includes Ti and DSC includes alloys
📝 SQL: SELECT * FROM entries 
WHERE 
    EXISTS (SELECT 1 FROM unnest(mat) AS val WHERE val ILIKE '%ti%') 
    AND EXISTS (SELECT 1 FROM unnest(dsc) AS val WHERE val ILIKE '%alloys%') 
LIMIT 10;


,id,index_id,abstract,pro,mat,smt,dsc,spl,cmt,apl,title,doi,pii,journal
0,7,6,"The wetting behaviour of Ag, Cu and Cu-Ti allo...","[['wetting behaviour', 'stoichiometry', 'hypos...","[['Ag', 'Cu', 'Cu-Ti', 'TiC', 'TiN', 'Cu', 'Ti...",[[]],"[['alloys', 'ceramic interfaces']]",[[]],"[['sessile drop method', 'dielectric continuum...",[[]],Wetting of titanium nitride and titanium carbi...,10.1016/1359-6454(95)00165-0,1359645495001650,Acta_Materialia
1,8,7,The present paper deals with the mechanism ope...,"[['structure', 'microstructure', 'stacking seq...","[['TiAl', 'Ll0']]",[[]],"[['two - phase', 'alloys', 'lamellar', 'alloys...","[['γ', 'α2', 'γ', 'α', 'A3', 'α 2', 'DO19', 'f...",[[]],[[]],Phase transformation mechanisms involved in tw...,10.1016/1359-6454(95)00167-4,1359645495001674,Acta_Materialia
2,9,8,This paper is directed towards examination of ...,"[['grain boundary migrations', 'structure', 'g...",[['TiAl']],[['rapid cooling']],"[['two - phase', 'alloys', 'interfaces', 'lame...","[['α', 'γ', 'γ', 'f.c.c.', 'α γ']]",[[]],[[]],Phase transformation mechanisms involved in tw...,10.1016/1359-6454(95)00168-6,1359645495001686,Acta_Materialia
3,46,45,Composite microstructures were grown from TiAl...,"[['microstructures', 'microstructure', 'compos...","[['TiAlSi', 'gamma', 'alpha2', 'Al', 'Si', 'Ti...","[['directional solidification techniques', 'me...","[['Composite', 'alloys', 'lamellar', 'lamellae...","[['alpha', 'alpha2', 'gamma', 'alpha2']]","[['tensile', 'three - point bend tests', 'bend']]",[[]],Directional solidification and microstructural...,10.1016/1359-6454(95)00338-x,135964549500338X,Acta_Materialia
4,53,52,The effect of grain size and titanium content ...,"[['grain size', 'titanium content', 'flow stre...","[['CuTi', 'Cu - 1.5 wt % Ti', 'Cu - 2.7 wt % T...","[['solution - treated', 'solid solution streng...","[['alloys', 'alloys', 'alloys', 'alloys', 'all...","[['body - centred tetragonal', 'β']]",[['Hall - Petch relation']],[[]],The alloy content and grain size dependence of...,10.1016/1359-6454(95)00358-4,1359645495003584,Acta_Materialia
5,732,731,A polycrystal undergoes microstructural change...,"[['grain - refining potency', 'intermetallic',...","[['Si', 'Al-Ti-B', 'Al-Si', 'Si', 'Ti', 'Ti5Si...",[[]],"[['casting alloys', 'surface', 'matrix', 'part...",[[]],"[['crystallographic investigation', 'edge - to...","[['grain refiners', 'coating']]",Spatial correlation of high-energy grain bound...,10.1016/j.actamat.2006.09.047,S1359645406006768,Acta_Materialia
6,73,72,The modes and controlling mechanisms of cavita...,"[['cavitation erosion', 'intermetallic', 'cavi...","[['TiAl', 'titanium aluminide', 'Ti52Al ( at. ...","[['plasma arc - melted', 'cavitation erosion',...","[['alloys', 'alloy', 'alloys', 'alloys']]",[[]],[[]],[[]],Mechanisms of cavitation erosion of TiAl-based...,10.1016/1359-6454(95)00419-x,135964549500419X,Acta_Materialia
7,75,74,x-phase particles are precipitated when Ti is ...,"[['shape memory', 'stability']]","[['Ti', 'CuAlNi', 'Mn', 'Mn']]",[['thermomechanical history']],"[['particles', 'alloys', 'precipitates', 'allo...",[[]],[[]],[['grain refiner']],A fundamental study of the χ-phase precipitati...,10.1016/1359-6454(95)00435-1,1359645495004351,Acta_Materialia
8,78,77,The massive α → γ\n m tran...,"[['γ m', 'γ', 'grain boundaries', 'gamma', 'γ ...","[['Ti49Al', 'Ti48Al2Nb2Mn', 'Ti55Al25Ta', 'Ti5...","[['solution heating', 'quenching', 'quenching'...","[['alloys', 'alloys', 'lamellae', 'lamellae', ...","[['α', 'γ', 'f.c.c.', 'L10', 'f.c.c.', 'L10', ...",[['analytical transmission electron microscopy...,[[]],The massive transformation in TiAl alloys: Me...,10.1016/1359-6454(95)00453-x,135964549500453X,Acta_Materialia
9,80,79,The general concept of the grain refinement me...,"[['grain refinement mechanism', 'stoichiometry']]","[['AlSi', 'silicon', 'AlSi', 'TiB2', 'AlSi', '...",[[]],"[['hypoeutectic', 'alloys', 'alloys', 'alloys'...","[['α-phase', 'α-Al']]",[[]],[['foundry']],Grain refinement mechanis

### Those abstracts are too long so let's drop those and paginate the results:

In [5]:
ask_agent("Find 10 entries where MAT includes Ti and DSC includes alloys")

🧠 Prompt: Find 10 entries where MAT includes Ti and DSC includes alloys
📝 SQL: SELECT index_id, title, doi, mat, dsc FROM entries WHERE EXISTS (SELECT 1 FROM unnest(mat) AS val WHERE val ILIKE '%ti%') AND EXISTS (SELECT 1 FROM unnest(dsc) AS val WHERE val ILIKE '%alloys%') LIMIT 10 OFFSET 0;


,index_id,title,doi,mat,dsc
0,6,Wetting of titanium nitride and titanium carbi...,10.1016/1359-6454(95)00165-0,"[['Ag', 'Cu', 'Cu-Ti', 'TiC', 'TiN', 'Cu', 'Ti...","[['alloys', 'ceramic interfaces']]"
1,7,Phase transformation mechanisms involved in tw...,10.1016/1359-6454(95)00167-4,"[['TiAl', 'Ll0']]","[['two - phase', 'alloys', 'lamellar', 'alloys..."
2,8,Phase transformation mechanisms involved in tw...,10.1016/1359-6454(95)00168-6,[['TiAl']],"[['two - phase', 'alloys', 'interfaces', 'lame..."
3,45,Directional solidification and microstructural...,10.1016/1359-6454(95)00338-x,"[['TiAlSi', 'gamma', 'alpha2', 'Al', 'Si', 'Ti...","[['Composite', 'alloys', 'lamellar', 'lamellae..."
4,52,The alloy content and grain size dependence of...,10.1016/1359-6454(95)00358-4,"[['CuTi', 'Cu - 1.5 wt % Ti', 'Cu - 2.7 wt % T...","[['alloys', 'alloys', 'alloys', 'alloys', 'all..."
5,731,Spatial correlation of high-energy grain bound...,10.1016/j.actamat.2006.09.047,"[['Si', 'Al-Ti-B', 'Al-Si', 'Si', 'Ti', 'Ti5Si...","[['casting alloys', 'surface', 'matrix', 'part..."
6,72,Mechanisms of cavitation erosion of TiAl-based...,10.1016/1359-6454(95)00419-x,"[['TiAl', 'titanium aluminide', 'Ti52Al ( at. ...","[['alloys', 'alloy', 'alloys', 'alloys']]"
7,74,A fundamental study of the χ-phase precipitati...,10.1016/1359-6454(95)00435-1,"[['Ti', 'CuAlNi', 'Mn', 'Mn']]","[['particles', 'alloys', 'precipitates', 'allo..."
8,77,The massive transformation in TiAl alloys: Me...,10.1016/1359-6454(95)00453-x,"[['Ti49Al', 'Ti48Al2Nb2Mn', 'Ti55Al25Ta', 'Ti5...","[['alloys', 'alloys', 'lamellae', 'lamellae', ..."
9,79,Grain refinement mechanisms of hypoeutectic Al...,10.1016/1359-6454(96)00021-3,"[['AlSi', 'silicon', 'AlSi', 'TiB2', 'AlSi', '...","[['hypoeutectic', 'alloys', 'alloys', 'alloys'..."
